# Part 2: Constructing SVD Music Recommender System

In [57]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import svds
import pandas as pd
from scipy.sparse import csr_matrix
import scipy.sparse
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [1]:
import os
os.getcwd()

'C:\\Users\\Ollie\\Documents'

In [2]:
triplets_df = pd.read_csv('triplets_df.csv')

In [3]:
song_df = pd.read_csv('songs_df.csv')

In [4]:
triplets_df['user_id'] = triplets_df['user_id'].astype('category')
triplets_df['song_id'] = triplets_df['song_id'].astype('category')

In [5]:
# Load in sparse matrix formed in Part 1
sparse_matrix = scipy.sparse.load_npz('sparse_matrix.npz')

In [6]:
# sparse_matrix needs to be of dtype float32 or 64
sparse_matrix.dtype

dtype('int64')

In [7]:
# Convert matrix to appropriate type
sparse_matrix = csr_matrix(sparse_matrix, dtype=float)

In [8]:
sparse_matrix = sparse_matrix.astype(np.float32)

In [9]:
sparse_matrix.dtype

dtype('float32')

In [10]:
# In order construct an SVD we need to decide on a k value for latent features 

In [11]:
U, sigma, Vt = svds(sparse_matrix, k=6000)

In [12]:
sigma = np.diag(sigma)

In [13]:
# When trying to predict across all users in the matrix we run into memory allocation issues
# One way around this is to only predict for a particular user, on a user by user basis
# SVD can still train on the whole dataset and get the latent features, meaning it's still drawing on all user-song interactions for predictions 

In [14]:
def predict_for_user(user_index, U, sigma, Vt):
    user_U = U[user_index, :]
    user_predictions = np.dot(np.dot(user_U, sigma), Vt)
    return user_predictions

# We will get prediction for the first user - user 0
user_predictions = predict_for_user(0, U, sigma, Vt)

In [15]:
user_predictions

array([ 0.00191561,  0.01210143,  0.00053172, ..., -0.00950143,
        0.00726082, -0.00486799], dtype=float32)

In [16]:
# Write function to generate recommendations from this users predictions 

In [17]:
def generate_recommendations(user_index, user_predictions, interaction_matrix, song_id_map, n=10):
   
    # Get items the user has already interacted with
    user_interacted_items = interaction_matrix[user_index].nonzero()[1]
    
    # Exclude already interacted items by setting their prediction scores to -np.inf
    user_predictions[user_interacted_items] = -np.inf
    
    # Get the indices of the top N recommendations, -n gives the last n but argsort is in reverse so use [::-1]
    top_n_indices = np.argsort(user_predictions)[-n:][::-1]
    
    # Map the indices back to item (song) IDs
    top_n_item_ids = [song_id_map[idx] for idx in top_n_indices]
    
    return top_n_item_ids

In [18]:
# For this function to work  we will need to pass in a mapping
# This is because the recommendation gives us top_n_indices, the indicies found in the sparce matrix
# These need to be mapped back to actual song_ids, then from song ids to titles

In [19]:
# -- Mapping Explanation --
# This creates a dictionary where the key is idx and song_id is original song_id from triplets_df
# This works because when the csr was programmed orginally, we did triplets_df['song_id'].cat.codes
# This assigns a unique code from 0 to each song_id. 
# Doing .cat.categories returns all these song_id in the order of their assigned codes, so enumerate is used to form this mapping
song_id_map = {idx: song_id for idx, song_id in enumerate(triplets_df['song_id'].cat.categories)}

In [20]:
# View dictionary mapping
song_id_map

{0: 'SOAAAGQ12A8C1420C8',
 1: 'SOAACPJ12A81C21360',
 2: 'SOAACSG12AB018DC80',
 3: 'SOAAEJI12AB0188AB5',
 4: 'SOAAFAC12A67ADF7EB',
 5: 'SOAAFYH12A8C13717A',
 6: 'SOAAJMQ12A6D4F7D17',
 7: 'SOAAKPM12A58A77210',
 8: 'SOAALWN12A6D4F7FDA',
 9: 'SOAAMOW12AB018149B',
 10: 'SOAAOYI12AB01831CE',
 11: 'SOAAROC12A6D4FA420',
 12: 'SOAARXR12A8C133D15',
 13: 'SOAATHE12A8C13ADD6',
 14: 'SOAATLI12A8C13E319',
 15: 'SOAAUKC12AB017F868',
 16: 'SOAAVUV12AB0186646',
 17: 'SOAAWEE12A6D4FBEC8',
 18: 'SOABGOB12A6701D1FA',
 19: 'SOABHNP12A8AE46E82',
 20: 'SOABHYV12A6D4F6D0F',
 21: 'SOABJBU12A8C13F63F',
 22: 'SOABJTC12A58A7DE0E',
 23: 'SOABNLP12A6D4F87F4',
 24: 'SOABNPW12A6D4FC9B5',
 25: 'SOABOHM12AB018509C',
 26: 'SOABOXV12AC3DF82F7',
 27: 'SOABPQU12A58A78441',
 28: 'SOABQTG12A6701F3DB',
 29: 'SOABRAB12A6D4F7AAF',
 30: 'SOABTTR12A6D4FC2EB',
 31: 'SOABXNE12A8C13B818',
 32: 'SOACBLB12AB01871C7',
 33: 'SOACDMD12A67AD8332',
 34: 'SOACERJ12A67AD865E',
 35: 'SOACGVR12A8C13B60A',
 36: 'SOACIPG12A8AE47E1C',
 37: 'SOACK

In [21]:
# Get recommendations for user 0 
top_recommendations = generate_recommendations(0, user_predictions, sparse_matrix, song_id_map, n=10)
top_recommendations

['SORPBQE12AF729D727',
 'SONBYCV12A58A7CCD7',
 'SOABNPW12A6D4FC9B5',
 'SOCAIGK12A8C143D86',
 'SODCAJF12AB018371E',
 'SOBHHHO12A58A78B94',
 'SOXQALX12AB0181468',
 'SORXWPW12A81C224BE',
 'SOHVJKX12A670208ED',
 'SOZZLTY12A67AE0AD0']

In [22]:
# Get song titles from songs_df
song_df[song_df['song_id'].isin(top_recommendations)]

,Unnamed: 0,song_id,title,release,artist_name,year,title_artist
207667,207714,SORPBQE12AF729D727,Old Yellow Bricks,Favourite Worst Nightmare,Arctic Monkeys,2007,Old Yellow Bricks - Arctic Monkeys
246797,246866,SOXQALX12AB0181468,Other Mathematics,Other Mathematics,Ex-Models,0,Other Mathematics - Ex-Models
251364,251435,SONBYCV12A58A7CCD7,One Last Kiss (LP Version),From Them_ Through Us_ To You,Madina Lake,0,One Last Kiss (LP Version) - Madina Lake
369652,369788,SOCAIGK12A8C143D86,Furious Rose,Firecracker,Lisa Loeb,1997,Furious Rose - Lisa Loeb
430486,430671,SOABNPW12A6D4FC9B5,The Canals Of Our City,The Gulag Orkestar,Beirut,2005,The Canals Of Our City - Beirut
539856,540161,SOHVJKX12A670208ED,For What It's Worth,Best Of,The Cardigans,2003,For What It's Worth - The Cardigans
579224,579563,SODCAJF12AB018371E,Once In A Lifetime,Sonic Firestorm,Dragonforce,2004,Once In A Lifetime - Dragonforce
935065,935909,SOZZLTY12A67AE0AD0,Ruska,Apocalyptica,Apocalyptica,2005,Ruska - Apocalyptica
937945,938796,SOBHHHO12A58A78B94,Jeremy,Ten,Pearl Jam,1991,Jeremy - Pearl Jam
944771,945630,SORXWPW12A81C224BE,All Fired Up,Our Love To Admire,Interpol,0,All Fired Up - Interpol


In [23]:
# Above shows, according to our data and SVD model, for user 0 their top 10 recommended songs that they have supposedly not listen to

In [24]:
# -- Evaluation of Model - RMSE --

In [25]:
# We will evaluate on user 0

In [26]:
# To evaluate this model we need to compare for this user, their original listen counts with the model's preictions

In [27]:
# Creating mapping from codes to user_id, from the csr matrix user to their actual user_id in the original df 
user_id_mapping = {code: user_id for code, user_id in enumerate(triplets_df['user_id'].cat.categories)}

In [28]:
original_user_id = user_id_mapping[0]
original_user_id

'00003a4459f33b92906be11abe0e93efc423c0ff'

In [29]:
# Identify this user's listen_counts

In [30]:
user_data = triplets_df[triplets_df['user_id'] == '00003a4459f33b92906be11abe0e93efc423c0ff']
true_listens = user_data.set_index('song_id')['listen_count']
true_listens = true_listens.sort_index()
true_listens

song_id
SOJJRVI12A6D4FBE49    1
SOKJWZB12A6D4F9487    4
SOMZHIH12A8AE45D00    3
SONFEUF12AAF3B47E3    3
SOVMGXI12AF72A80B0    1
SOVNPBK12A6D4F6A67    2
SOWVBDQ12A8C13503D    3
Name: listen_count, dtype: int64

In [31]:
# Predicted listens from SVD for all songs
predicted_listens = predict_for_user(0, U, sigma, Vt)
predicted_listens_series = pd.Series(predicted_listens, index=song_id_map.values())
predicted_listens_series

SOAAAGQ12A8C1420C8    0.001916
SOAACPJ12A81C21360    0.012101
SOAACSG12AB018DC80    0.000532
SOAAEJI12AB0188AB5    0.002232
SOAAFAC12A67ADF7EB   -0.021715
                        ...   
SOZZTNF12A8C139916   -0.004585
SOZZVWB12AB0189C30   -0.016654
SOZZWZV12A67AE140F   -0.009501
SOZZYAO12A6701FF36    0.007261
SOZZZPV12A8C1444B5   -0.004868
Length: 10000, dtype: float32

In [32]:
# Listen counts predicted by model for listened songs 
user_pred = predicted_listens_series[true_listens.index]
user_pred

song_id
SOJJRVI12A6D4FBE49    0.966738
SOKJWZB12A6D4F9487    2.079542
SOMZHIH12A8AE45D00    2.437272
SONFEUF12AAF3B47E3    1.940426
SOVMGXI12AF72A80B0    0.969132
SOVNPBK12A6D4F6A67    1.921473
SOWVBDQ12A8C13503D    3.072544
dtype: float32

In [ ]:
# On first glance a lot of these numbers are quite promising with the first one, and bottom three being practically spot on
# The others are only 1-1.5 off

In [33]:
# Calculate RMSE
mae = mean_absolute_error(true_listens, user_pred)
rmse = np.sqrt(mean_squared_error(true_listens, user_pred))
rmse

0.8569885408570102

In [34]:
triplets_df['listen_count'].describe()

count    1.973852e+06
mean     2.542916e+00
std      3.037282e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      3.000000e+00
max      2.200000e+01
Name: listen_count, dtype: float64

## Evaluation 

Interpreting this RMSE is not entirely straightforward. Given that we are predicting a listen count, being less than 1 off is not too bad. There are quite a few data points where listen counts are in the 10s, with the highest being 22; in such cases, being 0.85 off is good.

Moreover, the standard deviation is 3, meaning that listen counts vary around the mean quite a lot. Therefore, for the RMSE to be considerably less is a promising sign. In other words, it reduces the data's error to a level significantly lower than the typical variation.

Additionally, in a recommender system, predicting a listen count within 1 count of accuracy is probably not necessary for recommending a song. The recommendation doesn't need to know exactly how many listens there will be, just whether the user will like the song—i.e., if the listen count is above a certain threshold. For example, if the model predicts 5 listens when the actual count is 10, it is still effective in identifying that the user likes or listens to the song.

However, it's important to note that this RMSE can change with different users. For users with more songs listened to and listen counts closer to the average, the user's RMSE can be as low as 0.6 (as in the case of user 4). This makes sense because, generally speaking, having more data to predict from tends to lead to higher accuracy. It's also common for models to struggle to predict more extreme values, as they are mostly trained on average values.


In [36]:
pd.set_option('display.float_format', '{:.4f}'.format)

In [37]:
# The next stages will be to try and improve the model as well as get RMSE values for the whole matrix, not just on a user by user basis
# One way to do this is with a truncated svd

# Secondly, given we have RMSE that is low and that may continue to decrease with higher values of K, it's important to check if this is not just the model overfitting to the training data.
# If our model is trained on the whole matrix and performs well, it may not necessarily perform well on unseen data.
# So we will also use cross validation to analyse how well the model learns on unseen data as well as a holdout set that we can test against at the end

# -- Summary of Next stages -- 
# 1. Define holdout set
# 2. Get RMSE values over the whole matrix with Truncated SVD
# 3. Carry out cross validation
# 4. Test model against holdout set
# 5. Evaluation

In [68]:
# 1. Define holdout set
train_val_data, holdout_data = train_test_split(sparse_matrix, test_size=0.1, random_state=42)

In [97]:
# 2. Training and RMSE calculation on whole matrix (truncated SVD)
# NOTE: k = 1500 is the highest k value that computational resources allowed for in this case
k = 1500
svd = TruncatedSVD(n_components=k)
U = svd.fit_transform(train_val_data)
Vt = svd.components_
predicted_ratings = np.dot(U, Vt)

In [98]:
# For the RMSE calculation its essential that the index for the prediction and the train_val align properly
# We can test this with the steps below
non_zero_indices = train_val_data.nonzero()
actual_ratings = train_val_data.data
predicted_ratings = np.dot(U, Vt)
predicted_ratings_at_indices = predicted_ratings[non_zero_indices]

In [99]:
# Check the shapes are the same 
print(f"Actual Ratings Shape: {actual_ratings.shape}")
print(f"Predicted Ratings Shape: {predicted_ratings_at_indices.shape}")

Actual Ratings Shape: (1779590,)
Predicted Ratings Shape: (1779590,)


In [100]:
# Print the first few pairs, given the predictions are all close to the actual would suggest the indicies are aligned
for i in range(3):
    print(f"Actual: {actual_ratings[i]}, Predicted: {predicted_ratings_at_indices[i]}")

Actual: 1.0, Predicted: 1.0016509294509888
Actual: 2.0, Predicted: 1.8935208320617676
Actual: 3.0, Predicted: 2.9559836387634277


In [101]:
# print rmse
rmse = np.sqrt(mean_squared_error(train_val_data.data, predicted_ratings[train_val_data.nonzero()]))

In [102]:
rmse

2.015817

In [103]:
# 3. Cross validation

kf = KFold(n_splits=5) 
k = 1500 
results = []

for train_index, test_index in kf.split(train_val_data):
    train_data = train_val_data[train_index]
    test_data = train_val_data[test_index]

    # Run trucated SVD and get predictions and rmse as before but for each fold
    svd = TruncatedSVD(n_components=k)  
    U = svd.fit_transform(train_data)
    Vt = svd.components_
    predicted_ratings = np.dot(U, Vt)
    non_zero_indices = test_data.nonzero()
    actual_ratings = test_data.data
    predicted_ratings_at_indices = predicted_ratings[non_zero_indices]
    rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings_at_indices))
    results.append(rmse)

In [104]:
average_rmse = np.mean(results)
average_rmse

3.9633873

In [105]:
# 4. Test the model against the holdout set 
# As we are keeping k the same, we can now use the svd already trained, against the holdout set, to again see how the model performs on unseen data


TruncatedSVD(n_components=1500)

In [107]:
holdout_dense = holdout_data.toarray()
U_holdout = svd.transform(holdout_dense)

In [108]:
predicted_ratings_holdout = np.dot(U_holdout, Vt)

In [109]:
non_zero_indices = holdout_data.nonzero()
actual_ratings_holdout = holdout_data.data
predicted_ratings_at_indices_holdout = predicted_ratings_holdout[non_zero_indices]

In [110]:
holdout_rmse = np.sqrt(mean_squared_error(actual_ratings_holdout, predicted_ratings_at_indices_holdout))
print(f"Holdout RMSE: {holdout_rmse}")

Holdout RMSE: 2.344604253768921


# --- Final Evaluation --- 

We can see from comparing the RMSE on the training/validation data and the holdout set that the model still performs well on unseen data. This indicates that the model generalizes well and exhibits minimal overfitting, as the RMSE is only slightly higher (+0.3) for the unseen data set. An RMSE of 2.02 and 2.34 is quite good for this dataset, but this was achieved with a value of k only at 1500, as any higher value led to memory constraints.

Having already tested with a k value as high as 6000 earlier (not truncated), we know that the model's performance improved significantly with this increase. This makes sense because, with 2 million unique user-song pairs over 10,000 different songs, a higher k allows for capturing more latent features and patterns in the data. It's almost certain that the RMSE would have come down significantly for both the training data and the holdout test. The question is how much less the RMSE on the holdout set would decrease compared to the training set, or how much the higher k value would cause overfitting.

It's likely that the model would begin to overfit more at higher k values, so while the RMSE on the holdout set would decrease, it would probably not decrease as much as it would on the training/validation data. This is because, as k gets too large, it starts to fit the noise in the data.

Overall, it's unfortunate that computational constraints didn't allow for testing on higher k values, as it is highly likely both RMSEs would be lower. Additionally, this would have allowed for more extensive hyperparameter tuning to further optimize the model.

#### Addressing the 3.96 RMSE Cross-Validation Result
The cross-validation RMSE of 3.96 is quite high, suggesting that when the folds in the training/validation data are unseen, the model generally does not perform well. This 3.96 RMSE indicates more variation than the overall standard deviation. Cross-validation provides a rigorous evaluation of model performance by testing it on various unseen subsets of the data. The higher RMSE of 3.96 suggests that the model's performance is less consistent across different subsets.

#### There are several reasons why this might have occurred:

- Data Split: Depending on how the data is split, certain folds might contain users or items that are more challenging to predict, leading to a higher RMSE in those specific folds.
- Training Data Size: With smaller-sized folds, the model may need larger amounts of data to train effectively.
- Overfitting: If the original model (k = 1500) is slightly overfitting to the training/validation data, it might perform better on that data than on the more varied data in cross-validation.
- Noise Amplification: Cross-validation could amplify the effects of excessive noise in the dataset.

#### Improving the Cross-Validation RMSE Score

- Investigate whether specific folds are contributing disproportionately to the high RMSE.
- Examine the RMSE per fold to identify any outliers.
- Explore alternative methods for regularization, feature selection, or data preprocessing.
- Find workarounds for computational constraints and then cross-validate with higher k values.